In [190]:
import pandas as pd
import numpy as np
import keras
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

### Getting directory list

In [191]:
# f = Image.open('./fish/220px-Alligator_Gar_10.JPG.jpg', "r")
dirs = ['fish/' + x.strip(' ') + '/' for x in os.listdir('./fish/')]
temp = []

# Removing DS store
for d in dirs:
    if d == 'fish/.DS_Store':
        continue
    temp.append(d)
dirs = temp
dirs

['fish/Atlantic_Bonito/',
 'fish/Apache_Trout/',
 'fish/.DS_Store/',
 'fish/Alligator_Gar/',
 'fish/Atlantic_Cod/',
 'fish/Arctic_Char/',
 'fish/American_Shad/',
 'fish/Arctic_Grayling/',
 'fish/Almaco_Jack/',
 'fish/American_Eel/',
 'fish/Atlantic_Croaker/']

### Reading each image in a directory


In [216]:
d = 'fish/'


fish_data = {}

for directory in dirs:
    try:
        dir_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    except NotADirectoryError:
        continue
    
    images = []
    print("Start loading images for: " + directory)
    for image_file in dir_files:
        try:
            f = Image.open(directory + image_file)
            img_as_arr = np.array(f.resize((256,256)))
            images.append(img_as_arr)
            f.close()
        except OSError:
            continue
    fish_data[directory.split("/")[1]] = images



Start loading images for: fish/Atlantic_Bonito/
Start loading images for: fish/Apache_Trout/
Start loading images for: fish/Alligator_Gar/
Start loading images for: fish/Atlantic_Cod/
Start loading images for: fish/Arctic_Char/
Start loading images for: fish/American_Shad/
Start loading images for: fish/Arctic_Grayling/
Start loading images for: fish/Almaco_Jack/
Start loading images for: fish/American_Eel/
Start loading images for: fish/Atlantic_Croaker/


In [276]:
data = fish_data['Atlantic_Bonito']


samples = []
for image in data:
    if len(image.shape) != 3 or image.shape[2] != 3:
        continue
    samples.append(image)
    
shape = (len(samples),) + data[0].shape

print(shape)

x_placeholder = np.zeros(shape)

for i, sample in enumerate(samples):
    x_placeholder[i] = sample



# print(shape)
# fish_data['Atlantic_Bonito']
# print(shape)
# fish_data.keys()

(89, 256, 256, 3)
[[[[122.  36.  11.]
   [125.  39.  14.]
   [119.  40.   7.]
   ...
   [124.  81.  65.]
   [125.  82.  66.]
   [122.  79.  63.]]

  [[125.  40.  11.]
   [125.  40.  11.]
   [126.  43.  11.]
   ...
   [120.  77.  61.]
   [130.  87.  71.]
   [116.  73.  57.]]

  [[121.  37.   3.]
   [125.  41.   7.]
   [126.  39.   9.]
   ...
   [118.  75.  59.]
   [119.  76.  60.]
   [122.  79.  63.]]

  ...

  [[ 94.  22.   8.]
   [ 98.  21.   1.]
   [ 99.  26.   7.]
   ...
   [116.  65.  46.]
   [123.  66.  47.]
   [107.  50.  31.]]

  [[ 94.  21.  12.]
   [ 99.  22.   6.]
   [ 99.  26.   7.]
   ...
   [117.  71.  48.]
   [105.  53.  39.]
   [ 98.  36.  25.]]

  [[ 95.  24.   6.]
   [ 98.  22.   0.]
   [ 99.  26.   7.]
   ...
   [117.  74.  58.]
   [ 98.  63.  44.]
   [ 94.  37.  26.]]]


 [[[140. 141. 123.]
   [130. 133. 106.]
   [154. 155. 137.]
   ...
   [112. 115.  70.]
   [113. 119.  75.]
   [104. 109.  69.]]

  [[123. 124. 106.]
   [120. 122.  98.]
   [168. 170. 156.]
   ...
   

'Atlantic_Bonito'